# ACO-TR-LOGISTICS: Rota Optimizasyon Prototipi (Versiyon 2.0)
Bu not defteri, tüm düzeltmeleri ve en iyi pratikleri içeren, baştan sona çalışan bir prototiptir. Amacı, Çorlu bölgesi için belirlenen duraklara en verimli turu bulan bir rota optimizasyonu gerçekleştirmektir.

### Adım 1: Kurulum ve Kütüphaneleri Yükleme
Bu hücre, projenin çalışması için gerekli olan standart Python kütüphanelerini ve `src` klasörümüzdeki kendi yazdığımız `Ant` ve `ACOptimizer` sınıflarını import eder.

`sys.path` manipülasyonu, not defterimizin, bir üst dizindeki `src` klasörünü görmesini sağlamak için kritik bir adımdır.

In [ ]:
import sys
import os
import random

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
        sys.path.insert(0, project_root)

from src.aco import Ant, ACOptimizer

import osmnx as ox
import numpy as np
import folium

print("Kurulum tamamlandı. Kütüphaneler ve yerel modüller başarıyla import edildi.")

### Adım 2: Yol Ağını Çekme ve Hazırlama
Önceki hatalardan ders çıkardık. Algoritmanın tek yönlü yollarla kafasının karışmaması ve `KeyError` almamak için:
1. Önce standart 'drive' ağını (yönlü graf) çekeceğiz.
2. Sonra `ox.get_undirected()` fonksiyonu ile bu grafı **yönsüz (simetrik)** bir grafa çevireceğiz. Bu, eğer A'dan B'ye bir yol varsa, B'den A'ya da bir yol olduğunu garanti eder ve algoritmayı çok daha kararlı hale getirir.

In [ ]:
place_name = 'Çorlu, Tekirdağ, Türkiye'

G_directed = ox.graph_from_place(place_name, network_type='drive')

G = G_directed.to_undirected()

print(f"'{place_name}' için YÖNSÜZ yol ağı başarıyla çekildi ve 'G' nesnesine atandı.")
print(f"Toplam Düğüm: {len(G.nodes())}, Toplam Kenar: {len(G.edges())}")

### Adım 3: Optimizasyon Parametrelerini Ayarlama
Operasyon Şefi'nin strateji toplantısı! Algoritmamızın nasıl davranacağını belirleyen "ince ayar düğmelerini" burada tanımlıyoruz.

In [ ]:
ANT_COUNT = 10
ITERATIONS = 50
ALPHA = 1.0
BETA = 5.0
EVAPORATION_RATE = 0.5

### Adım 4: Problemi Tanımlama (Durakları Seçme
Karıncaların hangi problemi çözeceğini netleştirmeliyiz. Bu örnekte, haritadan rastgele **10 adet durak** seçeceğiz ve karıncalardan bu 10 durağın hepsine uğrayan en kısa turu bulmalarını isteyeceğiz. Bu, klasik bir **Gezgin Satıcı Problemi (TSP)** senaryosudur.

In [ ]:
all_graph_nodes = list(G.nodes())
nodes_to_visit = random.sample(all_graph_nodes, 10)
start_node = nodes_to_visit[0]

print(f"Toplam {len(nodes_to_visit)} adet durağa uğranacak.")
print(f"Başlangıç Durağı (Depo): {start_node}")

### Adım 5: Optimizasyonu Başlatma
Tüm hazırlıklar tamam. Şimdi `ACOptimizer` nesnesini oluşturup, `run` metodunu çağırarak optimizasyon sürecini başlatıyoruz. Bu işlem, bilgisayarınızın hızına bağlı olarak birkaç dakika sürebilir.

In [ ]:
ants = [Ant(G, start_node) for _ in range(ANT_COUNT)]
optimizer = ACOptimizer(
    graph=G,
    ants=ants,
    nodes_to_visit=nodes_to_visit,
    alpha=ALPHA,
    beta=BETA,
    evaporation_rate=EVAPORATION_RATE
)

print("Optimizasyon başlıyor...")
best_path, best_distance = optimizer.run(ITERATIONS)
print("\nOptimizasyon tamamlandı!")
print("--- SONUÇLAR ---")
print(f"En iyi bulunan tur mesafesi: {best_distance:.2f} metre")

### Adım 6: Sonucu Haritada Görselleştirme
Sayısal sonuç güzel, ama en tatmin edici kısım, bulunan en iyi rotayı interaktif bir harita üzerinde görmektir. `ox.plot_route_folium` fonksiyonu bu iş için mükemmeldir.

In [ ]:
import networkx as nx

if best_path and best_distance != float('inf'):
    print("Gerçekçi yol ağı rotası hesaplanıyor ve çiziliyor...")

    full_route_nodes = []
    for i in range(len(best_path) - 1):
        source = best_path[i]
        target = best_path[i+1]
        try:
            path_segment = nx.shortest_path(G, source=source, target=target, weight='length')
            full_route_nodes.extend(path_segment[:-1])
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    if best_path:
        full_route_nodes.append(best_path[-1])

    start_lat = G.nodes[start_node]['y']
    start_lon = G.nodes[start_node]['x']
    route_map = folium.Map(location=[start_lat, start_lon], zoom_start=13, tiles="OpenStreetMap")

    route_coordinates = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in full_route_nodes if node in G.nodes]
    folium.PolyLine(locations=route_coordinates, color='blue', weight=5, opacity=0.7).add_to(route_map)

    start_marker = folium.Marker(location=(start_lat, start_lon), icon=folium.Icon(color='green', icon='play'), popup='Başlangıç/Bitiş')
    stop_markers = []
    for node in nodes_to_visit:
        if node != start_node and node in G.nodes:
            stop_lat = G.nodes[node]['y']
            stop_lon = G.nodes[node]['x']
            marker = folium.Marker(location=(stop_lat, stop_lon), icon=folium.Icon(color='red', icon='info-sign'), popup=f'Durak: {node}')
            stop_markers.append(marker)
    
    start_marker.add_to(route_map)
    for marker in stop_markers:
        marker.add_to(route_map)

    display(route_map)
else:
    print("Bir rota bulunamadı veya rota hesaplanamadı.")